In [ ]:
import torch
from torch.utils.data import DataLoader
from torch.optim import SGD

import datasets
from datasets import load_dataset

import numpy as np

from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments

In [ ]:
device=('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
print(device)

In [ ]:
model_identifier='google-bert/bert-base-cased'

tokenizer=AutoTokenizer.from_pretrained(pretrained_model_name_or_path=model_identifier
                                      )
### loading 

dataset_name='yelp_review_full'

dataset=load_dataset(dataset_name)
num_label=np.unique(dataset['train']['label']).shape[0]
def tokenize_example(example):
    return (tokenizer(example['text'],padding=True,truncation=True)) ## it will radd features like input_ids,token_type_ids, attention_mask

In [ ]:
dataset

In [ ]:
dataset['train'].features

In [ ]:
import random
seed=101
dataset['train']=dataset['train'].shuffle(seed=seed).select(range(700))
dataset['test']=dataset['test'].shuffle(seed=seed).select(range(500))

In [ ]:
tokenized_dataset=dataset.map(tokenize_example,batched=True)

In [ ]:
tokenized_dataset=tokenized_dataset.remove_columns(['text'])

In [ ]:
tokenized_dataset=tokenized_dataset.rename_column("label","labels")

In [ ]:
tokenized_dataset.set_format('torch')

In [ ]:
train_dataset=tokenized_dataset['train']
test_dataset=tokenized_dataset['test']



In [ ]:
from torch.utils.data import Dataset

In [ ]:
### Now creating a dataloader 
batch_size=16
train_dataloader=DataLoader(dataset=train_dataset,
                            shuffle=True,
                            batch_size=batch_size)
test_dataloader=DataLoader(dataset=test_dataset,
                           shuffle=False,
                           batch_size=batch_size)

In [ ]:
## defining model 
model=AutoModelForSequenceClassification.from_pretrained(model_identifier,num_labels=num_label).to(device)

In [ ]:
## defining optimizer and lr_schedular
lr=1e-5
optimizer=torch.optim.AdamW(params=model.parameters(),lr=lr)

In [ ]:
## Creating training parameters 
epochs=3
num_training_steps=epochs*len(train_dataloader)

In [ ]:
from transformers import get_scheduler
lr_scheduler = get_scheduler(
    name="linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps
)

In [47]:
len(train_dataloader)

44

In [46]:
## time to train the model
from tqdm.auto import tqdm
progress_bar=tqdm(range(num_training_steps))
model.train()

for epoch in range(epochs):
    for step,batch in enumerate(train_dataloader):
        optimizer.zero_grad()
        batch={k:v.to(device) for k,v in batch.items()}
        outputs=model(**batch)
        loss=outputs.loss
        loss.backward()
        optimizer.step()
        lr_scheduler.step()
        print(f'Epoch: {epoch+1}|Steps: {(epoch+1)*(step+1)}|loss:{loss.item():.4f}')
        progress_bar.update(1)

  3%|▎         | 4/132 [01:18<41:41, 19.55s/it]


Epoch: 1|Steps: 1|loss:1.6797


Epoch: 1|Steps: 2|loss:1.6844


Epoch: 1|Steps: 3|loss:1.7292


Epoch: 1|Steps: 4|loss:1.7062


...

Epoch: 3|Steps: 120|loss:1.4650


Epoch: 3|Steps: 123|loss:1.5198


Epoch: 3|Steps: 126|loss:1.4388


Epoch: 3|Steps: 129|loss:1.3329


Epoch: 3|Steps: 132|loss:1.3388


In [70]:
import evaluate 
model.eval()
metrics=evaluate.load('accuracy')

for batch in test_dataloader:
    batch={k:v.to(device) for k,v in batch.items()}
    with torch.no_grad():
        outputs=model(**batch)

    logits=outputs.logits
    prediction=torch.argmax(logits,dim=-1)
    metrics.add_batch(predictions=prediction,references=batch['labels'])

metrics.compute()


{'accuracy': 0.402}